In [30]:
import requests
import pandas as pd
import json


In [31]:


# Fetch data from the API
def fetch_data():
    url = "https://api.i-colors.ru/ms/stocks/smart/now"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch data: {response.status_code}")

# Transform the data into a DataFrame
def transform_data(data):
    stocks = data.get('stocks', [])

    # Normalize the nested JSON structure
    df = pd.json_normalize(stocks)

    # Convert the entire DataFrame to strings to avoid JSON serialization issues
    df = df.applymap(lambda x: x.isoformat() if isinstance(x, pd.Timestamp) else x)

    # Serialize the DataFrame to JSON
    df['metadata_json'] = df.apply(lambda x: json.dumps(x.to_dict()), axis=1)

    return df

# Main execution
if __name__ == "__main__":
    try:
        data = fetch_data()
        df = transform_data(data)
        print(df)
    except Exception as e:
        print(e)


                  moment assortment.type assortment.code  \
0    2025-05-15 18:28:23         product      5K5A2WW_DM   
1    2025-05-15 18:28:23         product        KFE5T_DM   
2    2025-05-15 18:28:23         product   7500435205979   
3    2025-05-15 18:28:23         product        81570020   
4    2025-05-15 18:28:23         product        Б0049647   
..                   ...             ...             ...   
224  2025-05-15 18:28:23         product   7702018597840   
225  2025-05-15 18:28:23         product   7702018597826   
226  2025-05-15 18:28:23         product      15102024-3   
227  2025-05-15 18:28:23         product           усл49   
228  2025-05-15 18:28:23         product                   

                                       assortment.name assortment.gtin  \
0                       Венчик для миксеров KitchenAid   4673760580646   
1    Лопатка с гибким силиконовым ребром для миксер...   4673760580639   
2    Насадка для стрижки бороды Braun XT20 Black дл...   

C:\Users\User\AppData\Local\Temp\ipykernel_11448\3997910250.py:18: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.isoformat() if isinstance(x, pd.Timestamp) else x)


In [32]:
df

,moment,assortment.type,assortment.code,assortment.name,assortment.gtin,assortment.brand,assortment.state,assortment.folder,assortment.volume,stock.stock,stock.transit,stock.reserve,stock.quantity,stock.days,stock.cost,metadata_json
0,2025-05-15 18:28:23,product,5K5A2WW_DM,Венчик для миксеров KitchenAid,4673760580646,KitchenAid,Складская программа,БЫТОВАЯ ТЕХНИКА/Аксессуары для планетарных мик...,1.6,0,0,0,0,0.00,0.00,"{""moment"": ""2025-05-15 18:28:23"", ""assortment...."
1,2025-05-15 18:28:23,product,KFE5T_DM,Лопатка с гибким силиконовым ребром для миксер...,4673760580639,KitchenAid,Складская программа,БЫТОВАЯ ТЕХНИКА/Аксессуары для планетарных мик...,1.0,45,0,0,45,161.17,1646.79,"{""moment"": ""2025-05-15 18:28:23"", ""assortment...."
2,2025-05-15 18:28:23,product,7500435205979,Насадка для стрижки бороды Braun XT20 Black дл...,7500435205979,Braun,Складская программа,БЫТОВАЯ ТЕХНИКА/Техника для красоты/Аксессуары...,0.2,0,0,0,0,0.00,0.00,"{""moment"": ""2025-05-15 18:28:23"", ""assortment...."
3,2025-05-15 18:28:23,product,81570020,Сетка и режущий блок Braun Series 3 21B Black,4210201163596,Braun,Складская программа,БЫТОВАЯ ТЕХНИКА/Техника для красоты/Аксессуары...,0.3,20,0,0,20,9.12,8687.30,"{""moment"": ""2025-05-15 18:28:23"", ""assortment...."
4,2025-05-15 18:28:23,product,Б0049647,Сетка и режущий блок Braun Series 5 53B Black,4210201262886,Braun,Складская программа,БЫТОВАЯ ТЕХНИКА/Техника для красоты/Аксессуары...,0.3,8,0,0,8,9.12,15915.64,"{""moment"": ""2025-05-15 18:28:23"", ""assortment...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224,2025-05-15 18:28:23,product,7702018597840,Подарочный набор с женской бритвой Gillette Ve...,7702018597840,Venus GILLETTE,Складская программа,КРАСОТА И ГИГИЕНА/Бритвы и лезвия/Подарочные б...,1.2,0,0,0,0,0.00,0.00,"{""moment"": ""2025-05-15 18:28:23"", ""assortment...."
225,2025-05-15 18:28:23,product,7702018597826,Подарочный набор с женской бритвой Gillette Ve...,7702018597826,Venus GILLETTE,Складская программа,КРАСОТА И ГИГИЕНА/Бритвы и лезвия/Подарочные б...,1.2,0,0,0,0,0.00,0.00,"{""moment"": ""2025-05-15 18:28:23"", ""assortment...."
226,2025-05-15 18:28:23,product,15102024-3,логистика фасовка,,,,,0.0,0,0,0,0,0.00,0.00,"{""moment"": ""2025-05-15 18:28:23"", ""assortment...."
227,2025-05-15 18:28:23,product,усл49,Недостача склад,,,,,0.0,0,0,0,0,0.00,0.00,"{""moment"": ""2025-05-15 18:28:23"", ""assortment...."


In [33]:
import os
import pandas as pd
from clickhouse_connect import get_client

# Fetch ClickHouse password from environment variables
password = os.getenv('ClickHouse')

# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                                   # Your ClickHouse username
    password=password,                                  # Your ClickHouse password
    database='user1',                                   # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification
)

# Ensure date columns are in the correct format for ClickHouse
df['moment'] = pd.to_datetime(df['moment'])  # Convert to datetime

# Debugging: Check the data types of the DataFrame
print("Data types of df:")
print(df.dtypes)

# Rename columns to match ClickHouse table
df = df.rename(columns={
    'assortment.type': 'assortment_type',
    'assortment.code': 'assortment_code',
    'assortment.name': 'assortment_name',
    'assortment.gtin': 'assortment_gtin',
    'assortment.brand': 'assortment_brand',
    'assortment.state': 'assortment_state',
    'assortment.folder': 'assortment_folder',
    'assortment.volume': 'assortment_volume',
    'stock.stock': 'stock_stock',
    'stock.transit': 'stock_transit',
    'stock.reserve': 'stock_reserve',
    'stock.quantity': 'stock_quantity',
    'stock.days': 'stock_days',
    'stock.cost': 'stock_cost'
})

# Ensure the DataFrame has the correct columns
columns = [
    'moment', 'assortment_type', 'assortment_code', 'assortment_name',
    'assortment_gtin', 'assortment_brand', 'assortment_state',
    'assortment_folder', 'assortment_volume', 'stock_stock',
    'stock_transit', 'stock_reserve', 'stock_quantity',
    'stock_days', 'stock_cost', 'metadata_json'
]

# Reorder columns to match the expected order
df_copy = df[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in df_copy.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'stocks'

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")


Data types of df:
moment               datetime64[ns]
assortment.type              object
assortment.code              object
assortment.name              object
assortment.gtin              object
assortment.brand             object
assortment.state             object
assortment.folder            object
assortment.volume           float64
stock.stock                   int64
stock.transit                 int64
stock.reserve                 int64
stock.quantity                int64
stock.days                  float64
stock.cost                  float64
metadata_json                object
dtype: object
Sample data to insert: [(Timestamp('2025-05-15 18:28:23'), 'product', '5K5A2WW_DM', 'Венчик для миксеров KitchenAid', '4673760580646', 'KitchenAid', 'Складская программа', 'БЫТОВАЯ ТЕХНИКА/Аксессуары для планетарных миксеров', 1.6, 0, 0, 0, 0, 0.0, 0.0, '{"moment": "2025-05-15 18:28:23", "assortment.type": "product", "assortment.code": "5K5A2WW_DM", "assortment.name": "\\u0412\\u0435\\u043